In [1]:
import pandas as pd

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers


/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
test_data = pd.read_csv('test_data.csv')
train_data = pd.read_csv('train_data.csv')
train_labels = pd.read_csv('train_labels.csv')

In [3]:
del train_data['is_duplicate']

In [4]:
train_data_labels = pd.merge(train_data,train_labels, how='left')

In [5]:
train_data_labels = train_data_labels.dropna()

In [6]:
train_data_labels['question1'] = train_data_labels['question1'].str.replace('[^\w\s]','')
train_data_labels['question2']= train_data_labels['question2'].str.replace('[^\w\s]','')

In [10]:
train_data_labels.head()

,id,question1,question2,is_duplicate
0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,What is the story of Kohinoor KohiNoor Diamond,What would happen if the Indian government sto...,0
2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,Why am I mentally very lonely How can I solve it,Find the remainder when math2324math is divide...,0
4,4,Which one dissolve in water quikly sugar salt ...,Which fish would survive in salt water,0


In [11]:
from nltk.stem import SnowballStemmer
snow = SnowballStemmer('english')

train_data_labels['question1'] = train_data_labels['question1'].apply(lambda x: " ".join([snow.stem(word) for word in x.split()]))
train_data_labels['question2'] = train_data_labels['question2'].apply(lambda x: " ".join([snow.stem(word) for word in x.split()]))

In [12]:
train_data_labels.head()

,id,question1,question2,is_duplicate
0,0,what is the step by step guid to invest in sha...,what is the step by step guid to invest in sha...,0
1,1,what is the stori of kohinoor kohinoor diamond,what would happen if the indian govern stole t...,0
2,2,how can i increas the speed of my internet con...,how can internet speed be increas by hack thro...,0
3,3,whi am i mental veri lone how can i solv it,find the remaind when math2324math is divid by...,0
4,4,which one dissolv in water quik sugar salt met...,which fish would surviv in salt water,0


In [14]:
import re
train_data_labels['question1'] = train_data_labels['question1'].apply(lambda x: re.sub(r"n't", " not ", x))
train_data_labels['question2'] = train_data_labels['question2'].apply(lambda x: re.sub(r"n't", " not ", x))

In [15]:
train_data_labels['question1'] = train_data_labels['question1'].apply(lambda x:re.sub(r"\'ve", " have ", x))
train_data_labels['question1'] = train_data_labels['question1'].apply(lambda x:re.sub(r"can't", "cannot ", x))
train_data_labels['question1'] = train_data_labels['question1'].apply(lambda x:re.sub(r"i'm", "i am ", x))
train_data_labels['question1'] = train_data_labels['question1'].apply(lambda x:re.sub(r"\'re", " are ", x))
train_data_labels['question1'] = train_data_labels['question1'].apply(lambda x:re.sub(r"\'d", " would ", x))
train_data_labels['question1'] = train_data_labels['question1'].apply(lambda x:re.sub(r"\'ll", " will ", x))

In [16]:
train_data_labels['question2'] = train_data_labels['question2'].apply(lambda x:re.sub(r"\'ve", " have ", x))
train_data_labels['question2'] = train_data_labels['question2'].apply(lambda x:re.sub(r"can't", "cannot ", x))
train_data_labels['question2'] = train_data_labels['question2'].apply(lambda x:re.sub(r"i'm", "i am ", x))
train_data_labels['question2'] = train_data_labels['question2'].apply(lambda x:re.sub(r"\'re", " are ", x))
train_data_labels['question2'] = train_data_labels['question2'].apply(lambda x:re.sub(r"\'d", " would ", x))
train_data_labels['question2'] = train_data_labels['question2'].apply(lambda x:re.sub(r"\'ll", " will ", x))

In [17]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [18]:
train_data_labels['question1'] = train_data_labels['question1'].str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in stop))
train_data_labels['question2'] = train_data_labels['question2'].str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in stop))

In [19]:
train_data_labels['question1'] = train_data_labels['question1'].apply(lambda x: " ".join(x.lower() for x in x.split()))
train_data_labels['question2'] = train_data_labels['question2'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [20]:
from nltk.tokenize import word_tokenize
import nltk


train_data_labels['question1'] = train_data_labels['question1'].apply(word_tokenize)
train_data_labels['question2'] = train_data_labels['question2'].apply(word_tokenize)

In [21]:
train_data_labels.head()

,id,question1,question2,is_duplicate
0,0,"[step, step, guid, invest, share, market, india]","[step, step, guid, invest, share, market]",0
1,1,"[stori, kohinoor, kohinoor, diamond]","[would, happen, indian, govern, stole, kohinoo...",0
2,2,"[increas, speed, internet, connect, use, vpn]","[internet, speed, increas, hack, dns]",0
3,3,"[whi, mental, veri, lone, solv]","[find, remaind, math2324math, divid, 2423]",0
4,4,"[one, dissolv, water, quik, sugar, salt, metha...","[fish, would, surviv, salt, water]",0


In [22]:
question1 = train_data_labels['question1'].tolist()

In [23]:
question2 = train_data_labels['question2'].tolist()

In [24]:
questions = question1 + question2

In [25]:
import gensim 

In [26]:
model = gensim.models.Word2Vec(questions, min_count=1,size=100,workers=4)

In [27]:
model.save('QuoraGensim_v1.bin')